In [2]:
import pandas as pd
import numpy as np

In [3]:
games_details= pd.read_csv('Data/games_details.csv')
games=pd.read_csv('Data/games.csv')
players=pd.read_csv('Data/players.csv')
ranking=pd.read_csv('Data/ranking.csv')
teams=pd.read_csv('Data/teams.csv')

C:\Users\Prince\AppData\Local\Temp\ipykernel_30576\4257920790.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details= pd.read_csv('Data/games_details.csv')


In [4]:
games[['GAME_ID', 'SEASON', 'PTS_home', 'PTS_away']].isna().sum()


GAME_ID      0
SEASON       0
PTS_home    99
PTS_away    99
dtype: int64

In [5]:
games['TOTAL_PTS']=games['PTS_home']+games['PTS_away']

In [6]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,TOTAL_PTS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1,243.0
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1,232.0
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1,220.0
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1,206.0
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0,218.0


In [7]:
season_scoring=(games
    .groupby('SEASON')
    .agg(
        GAMES_PLAYED=('GAME_ID', 'nunique'),
        TOTAL_POINTS=('TOTAL_PTS', 'sum'),
        AVG_POINTS_PER_GAME=('TOTAL_PTS', 'mean')
    )
    .reset_index()
    .sort_values('SEASON')
    )

In [8]:
season_scoring['YOY_CHANGE'] = (
    season_scoring['AVG_POINTS_PER_GAME']
    .diff()
)


In [9]:
season_scoring.head()

,SEASON,GAMES_PLAYED,TOTAL_POINTS,AVG_POINTS_PER_GAME,YOY_CHANGE
0,2003,1385,239196.0,186.000000,NaN
1,2004,1362,264210.0,193.986784,7.986784
2,2005,1432,277205.0,193.578911,-0.407874
3,2006,1419,279250.0,196.793517,3.214606
4,2007,1411,280539.0,198.822821,2.029304


In [10]:
season_scoring.tail()

,SEASON,GAMES_PLAYED,TOTAL_POINTS,AVG_POINTS_PER_GAME,YOY_CHANGE
15,2018,1378,305733.0,221.867199,9.721757
16,2019,1241,276422.0,222.741338,0.874139
17,2020,1220,279293.0,223.613291,0.871953
18,2021,1389,306156.0,220.414687,-3.198604
19,2022,542,122233.0,225.522140,5.107453


In [11]:
season_scoring.to_csv('Output/season_scoring.csv', index=False)


In [16]:
home_games= games[['GAME_ID', 'SEASON', 'HOME_TEAM_ID', 'PTS_home']].rename(
    columns={'HOME_TEAM_ID': 'TEAM_ID','PTS_home': 'PTS'}
)

away_games= games[['GAME_ID', 'SEASON', 'VISITOR_TEAM_ID', 'PTS_away']].rename(
    columns={'VISITOR_TEAM_ID': 'TEAM_ID','PTS_away': 'PTS'}
)

team_games=pd.concat([home_games, away_games], ignore_index=True)

In [17]:
team_season_scoring=(
    team_games
    .groupby(['SEASON', 'TEAM_ID'])
    .agg(
        GAMES_PLAYED=('GAME_ID', 'nunique'),
        TOTAL_POINTS=('PTS', 'sum'),
        AVG_POINTS_PER_GAME=('PTS', 'mean')
    )
    .reset_index()
)

In [18]:
team_season_scoring= team_season_scoring.merge(
    teams[['TEAM_ID', 'NICKNAME']],
    on='TEAM_ID',
    how='left'
)

In [19]:
team_season_scoring.to_csv('Output/team_season_scoring.csv', index=False)

In [20]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,TOTAL_PTS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1,243.0
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1,232.0
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1,220.0
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1,206.0
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0,218.0


In [23]:
home_away_season=(
    games
    .groupby('SEASON')
    .agg(
        HOME_GAMES=('GAME_ID', 'nunique'),
        HOME_WINS=('HOME_TEAM_WINS', 'sum'),
        AWAY_WINS=('HOME_TEAM_WINS', lambda x: (1-x).sum()),
        AVG_HOME_PTS=('PTS_home', 'mean'),
        AVG_AWAY_PTS=('PTS_away', 'mean')
    )
    .reset_index()
)

In [24]:
home_away_season['HOME_WIN_PCT']= (home_away_season['HOME_WINS']/home_away_season['HOME_GAMES'])

In [25]:
home_away_season['AWAY_WIN_PCT'] = (
    home_away_season['AWAY_WINS'] / home_away_season['HOME_GAMES']
)

In [26]:
home_away_season.to_csv('Output/home_away_season.csv', index=False)

In [30]:
player_game_stats = games_details[
    games_details['MIN'].notna() &
    (games_details['MIN'] != '0') &
    (games_details['MIN'] != '0:00')
]

In [31]:
player_with_season= player_game_stats.merge(
    games[['GAME_ID', 'SEASON']],
    on='GAME_ID',
    how='left'
)

In [32]:
player_with_season[['GAME_ID', 'SEASON']].isna().sum()


GAME_ID    0
SEASON     0
dtype: int64

In [33]:
player_season_stats = (
    player_with_season
    .groupby(['SEASON', 'PLAYER_ID', 'PLAYER_NAME'])
    .agg(
        GAMES_PLAYED=('GAME_ID', 'nunique'),
        TOTAL_PTS=('PTS', 'sum'),
        TOTAL_AST=('AST', 'sum'),
        TOTAL_REB=('REB', 'sum'),
        FGM=('FGM', 'sum'),
        FGA=('FGA', 'sum'),
        FG3M=('FG3M', 'sum'),
        FG3A=('FG3A', 'sum'),
        FTM=('FTM', 'sum'),
        FTA=('FTA', 'sum')
    )
    .reset_index()
)


In [34]:
player_season_stats['PTS_PER_GAME'] = (
    player_season_stats['TOTAL_PTS'] / player_season_stats['GAMES_PLAYED']
)

player_season_stats['FG_PCT'] = (
    player_season_stats['FGM'] / player_season_stats['FGA']
)

player_season_stats['FG3_PCT'] = (
    player_season_stats['FG3M'] / player_season_stats['FG3A']
)

player_season_stats['FT_PCT'] = (
    player_season_stats['FTM'] / player_season_stats['FTA']
)


In [35]:
player_season_stats.replace([float('inf')], 0, inplace=True)


In [37]:
player_season_stats.to_csv(
    'Output/player_season_stats.csv',
    index=False
)


In [38]:
ranking[['TEAM_ID', 'SEASON_ID', 'CONFERENCE']].head()


,TEAM_ID,SEASON_ID,CONFERENCE
0,1610612743,22022,West
1,1610612763,22022,West
2,1610612740,22022,West
3,1610612756,22022,West
4,1610612746,22022,West


In [42]:
ranking.head()

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN


In [39]:
conference_season_stats = (
    ranking
    .groupby(['SEASON_ID', 'CONFERENCE'])
    .agg(
        TEAMS=('TEAM_ID', 'nunique'),
        TOTAL_GAMES=('G', 'sum'),
        TOTAL_WINS=('W', 'sum'),
        TOTAL_LOSSES=('L', 'sum'),
        AVG_WIN_PCT=('W_PCT', 'mean')
    )
    .reset_index()
)


In [47]:
conference_season_stats.head()

,SEASON_ID,CONFERENCE,TEAMS,TOTAL_GAMES,TOTAL_WINS,TOTAL_LOSSES,AVG_WIN_PCT
0,12005,East,15,114,53,61,0.467200
1,12005,West,15,114,61,53,0.532200
2,12006,East,15,116,58,58,0.500467
3,12006,West,15,116,62,54,0.536867
4,12007,East,15,99,51,48,0.511333


In [43]:
ranking_played = ranking[ranking['G'] > 0]


In [44]:
ranking_final = (
    ranking_played
    .sort_values('STANDINGSDATE')
    .groupby(['SEASON_ID', 'TEAM_ID'])
    .tail(1)
)


In [45]:
conference_season_stats = (
    ranking_final
    .groupby(['SEASON_ID', 'CONFERENCE'])
    .agg(
        TEAMS=('TEAM_ID', 'nunique'),
        TOTAL_GAMES=('G', 'sum'),
        TOTAL_WINS=('W', 'sum'),
        TOTAL_LOSSES=('L', 'sum'),
        AVG_WIN_PCT=('W_PCT', 'mean')
    )
    .reset_index()
    .sort_values('SEASON_ID')
)


In [46]:
conference_season_stats.head(6)
conference_season_stats[['TOTAL_GAMES', 'TOTAL_WINS']].min()


TOTAL_GAMES    30
TOTAL_WINS     14
dtype: int64

In [50]:
conference_season_stats['SEASON_YEAR'] = (
    conference_season_stats['SEASON_ID']
    .astype(str)
    .str[-4:]
    .astype(int)
)


In [51]:
conference_season_stats = (
    conference_season_stats
    .groupby(['SEASON_YEAR', 'CONFERENCE'], as_index=False)
    .agg(
        TOTAL_WINS=('TOTAL_WINS', 'sum'),
        AVG_WIN_PCT=('AVG_WIN_PCT', 'mean')
    )
)


In [52]:
conference_season_stats.to_csv(
    'Output/conference_season_stats.csv',
    index=False
)
